In [24]:
# 下载某个导演的电影数据集
from efficient_apriori import apriori
from lxml import etree
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import csv

In [28]:
driver = webdriver.Chrome(ChromeDriverManager().install())
# 设置想要下载的导演数据集
director = u'宁浩'
# 写入 CSV 文件
file_name = './'  + 'ninghao.csv'
base_url = 'https://movie.douban.com/subject_search?search_text='+director+'&cat=1002&start='
out = open(file_name, newline='', encoding='utf-8-sig')
csv_write = csv.writer(out, dialect='excel')
flags = []

# 下载指定页面数据
def download(request_url):
    driver.get(request_url)
    time.sleep(1)
    html = driver.find_element_by_xpath('//*').get_attribute('outerHTML')
    html = etree.HTML(html)
    # 设置电影名称、导演、演员的 XPATH
    movie_lists = html.xpath("/html/body/div[@id='wrapper']/div[@id='root']/div[1]//div[@class='item-root']/div[@class='detail']/div[@class='title']/a[@class='title-text']") 
    name_lists = html.xpath("/html/body/div[@id='wrapper']/div[@id='root']/div[1]//div[@class='item-root']/div[@class='detail']/div[@class='meta abstract_2']")
    # 获取返回的数据个数
    num = len(movie_lists)
    if num > 15:
        movie_lists = movie_lists[1:]
        name_lists = name_lists[1:]
    for (movie, name_list) in zip(movie_lists, name_list):
        # 会存在数据为空的情况
        if name_list.text is None:
            continue
        # 显示下演员名称
        print(name_list.text)
        names = name_list.text.split('/')
        # 判断导演是否为指定的 director
        if names[0].strip() == director and movie.text not in flags:
            # 将第一个字段设置为电影名称
            names[0] = movie.text
            flags.append(movie.text)
            csv_write.writerow(names)
    print('OK')
    print(num)
    if num >= 14:
        # 继续下一页
        return True
    else:
        # 没有下一页
        return False

# 开始的 ID 为 0、每页增加 15
start = 0
while start < 10000:
    request_url = base_url + str(start)
    flag = download(request_url)
    if flag:
        start = start + 15
    else:
        break
out.close()
print('finished')

/var/folders/gl/k85fznm93rjg0hmcn_57s_980000gq/T/ipykernel_24185/2714047961.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


AttributeError: 'WebDriver' object has no attribute 'find_element_by_xpath'

In [ ]:
lists = csv.reader(file_name, 'r', encoding='utf-8-sig')
# 数据加载
data = []
for names in lists:
    name_new = []
    for name in names:
        # 去掉演员数据中的空格
        name_new.append(name.strip())
    data.append(name_new[1:])
# 挖掘频繁项集和关联规则
itemsets, rules = apriori(data, min_support=0.5, min_confidence=1)
print(itemsets)
print(rules)